
# now let's initialize the list of reference point
ref_points = [[[]]*2 for i in range(6)]
try:
    with open('ref_points.json', 'r') as file:
        ref_points = json.load(file)
except RuntimeError as e:
    print("Error: ", e)
draw = False
grid_num = 0
def shape_selection(event, x, y, flags, param):
    # grab references to the global variables
    global ref_points, crop, draw
    ref_point = ref_points[grid_num]
    # ix, iy = (0, 0)
    # if the left mouse button was clicked, record the starting
    # (x, y) coordinates and indicate that cropping is being performed
    if event == cv2.EVENT_LBUTTONDOWN:
        draw = True
        ref_point[0] = (x, y)
        ref_point[1] = ()
        # image = clone.copy()
        # cv2.imshow("image", image)

    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates and indicate that
        # the cropping operation is finished
        if draw:
            draw = False
            # print(ref_point)
            ref_point[1] = (x, y)
            ix, iy = (ref_point[0][0], ref_point[0][1])
            # calculate the side length of the square
            side_length = max(abs(x - ix), abs(y - iy))
            # determine the bottom-right corner of the square
            if x >= ix and y >= iy:
                ref_point[1] = (ix + side_length, iy + side_length)
            elif x >= ix and y < iy:
                ref_point[1] = (ix + side_length, iy - side_length)
            elif x < ix and y >= iy:
                ref_point[1] = (ix - side_length, iy + side_length)
            else:
                ref_point[1] = (ix - side_length, iy - side_length)
            # cv2.imshow("image", image)




# load the image, clone it, and setup the mouse callback function
ret_val, image = cam.read()
# image = cv2.imread('download.png')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", shape_selection)

croped_img = None
# keep looping until the 'q' key is pressed
rotation = 2
prev_state = None
img_counter = 0
n_imgs = 1
croped_imgs = []
while True:
    # display the image and wait for a keypress
    ret_val, image = cam.read()
    for _ in range(rotation):
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    ref_point = ref_points[grid_num]
    if len(ref_point[1])==2:
        ix, iy = (ref_point[0][0], ref_point[0][1])
        x, y = (ref_point[1][0], ref_point[1][1])
        # calculate the side length of the square
        side_length = max(abs(x - ix), abs(y - iy))
        if side_length >= 64:
            # draw a square around the region of interest
            thickness = 2
            cv2.rectangle(image, 
                    (ref_point[0][0] - thickness, ref_point[0][1] - thickness), 
                    (ref_point[1][0] + thickness, ref_point[1][1] + thickness), 
                    (0, 255, 0), thickness)
            croped_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            croped_img = croped_img[ref_point[0][1]:ref_point[1][1], ref_point[0][0]:ref_point[1][0]]
            croped_img = cv2.resize(croped_img, (64, 64))
            croped_img = cv2.adaptiveThreshold(croped_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11,2)
            croped_img = 255-croped_img
            croped_img = croped_img.astype(np.float64)
            croped_img /= 255

            clear_output(wait=True)
            cv2.imshow("cropped", cv2.resize(croped_img, (256, 256)))
            
    
    cv2.imshow("image", image)
    
    key = cv2.waitKey(1) & 0xFF

    # press 'r' to reset the window
    if key == ord("r"):
        # draw = True
        rotation += 1
        rotation %= 4
        # image = clone.copy()
    elif key >= ord("1") and (key - ord("1")) < 6:
        grid_num_to_print = key - ord("1") + 1
        print(grid_num_to_print)
        grid_num = key - ord("1")
    # if the 'c' key is pressed, break from the loop
    elif key == 27:
        break

if len(ref_points)==6:
    import json
    with open('ref_points.json', 'w') as json_file:
        json.dump(ref_points, json_file) 
# close all open windows
print(ref_point)
cv2.destroyAllWindows()

In [44]:
import cv2
from IPython.display import clear_output
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from utils import ones_state, vis, state_ones
import serial
import time
cam = cv2.VideoCapture(1)

In [2]:
from detector import Detector
d = Detector(64, 64, "weights-00-0.28.hdf5")

model created with # params: 507858
loading model weights from path: weights-00-0.28.hdf5
loaded weights


c:\Users\aabdo\miniconda3\envs\tic_tac_toe\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
COMP_Turn_msg = "Computer turn!"
HUM_Turn_msg = "Your turn!"
win_msg = "You WON!!"
lose_msg = "You lost!"
draw_msg = "DRAW!"
def show_grid(c_origins, h_origins, c_char = 'O', message="",future_move=None):
    #messages...
    global COMP_Turn_msg
    global HUM_Turn_msg
    global win_msg
    global lose_msg
    global draw_msg
    assert len(c_origins) + len(h_origins) <= 9 , "sum of origins should be less than 9"
    # Create a blank white image (you can adjust the size as needed)
    message_hight = 100
    width, height = 300, 300
    img = np.ones((height+message_hight, width, 3), dtype=np.uint8) * 255

    # Draw horizontal lines
    for i in range(1, 3):
        y = i * height // 3
        cv2.line(img, (0, y), (width, y), (0, 0, 0), 2)

    # Draw vertical lines
    for i in range(1, 3):
        x = i * width // 3
        cv2.line(img, (x, 0), (x, height), (0, 0, 0), 2)
    # setup text
    font = cv2.FONT_HERSHEY_SIMPLEX
    h_char = "X" if c_char == "O" else "O"
    scale = 2
    thickness = 3
    # get boundary of this text
    h_textsize = cv2.getTextSize(h_char, font, scale, thickness)[0]
    # get coords based on boundary
    for h_org in h_origins:
        factor = ((h_org[0]*2-1),(h_org[1]*2-1))
        x = factor[0]*width // 6
        y = factor[1]*height // 6 
        textX = x - h_textsize[0]// 2
        textY = y + h_textsize[1]// 2

        # add text centered on image
        cv2.putText(img, h_char, (textX, textY ), font, scale, (0, 0, 255), thickness)

    # get coords based on boundary
    c_textsize = cv2.getTextSize(c_char, font, scale, thickness)[0]
    for c_org in c_origins:
        factor = ((c_org[0]*2-1),(c_org[1]*2-1))
        x = factor[0]*width // 6
        y = factor[1]*height // 6 
        textX = x - c_textsize[0]// 2
        textY = y + c_textsize[1]// 2

        # add text centered on image
        cv2.putText(img, c_char, (textX, textY ), font, scale, (0, 255, 0), thickness)
    #print future move
    if future_move != None and future_move not in c_origins:
        factor = ((future_move[0]*2-1),(future_move[1]*2-1))
        x_future = factor[0]*width // 6
        y_future = factor[1]*height // 6 
        textX = x_future - c_textsize[0]// 2
        textY = y_future + c_textsize[1]// 2

        # add text centered on image
        cv2.putText(img, c_char, (textX, textY ), font, scale, (255, 0, 0), thickness)
    ###
    if message != "":
        msg_color = (0,0,0)
        if message == HUM_Turn_msg:
            msg_color = (0,0,100)
        elif message == COMP_Turn_msg:
            msg_color = (0,100,0)
        elif message == win_msg:
            msg_color = (108,240,255)#fff06c
        elif message == lose_msg:
            msg_color = (48,17,76)#4c1130
        elif  message == draw_msg:
            msg_color = (0, 0, 0)
        msg_scale = 1
        msg_thickness = 2
        msg_font = cv2.FONT_HERSHEY_SIMPLEX
        msg_textsize = cv2.getTextSize(message, msg_font, msg_scale, msg_thickness)[0]
        msgY = 350
        msgX = (width-msg_textsize[0])//2
        cv2.putText(img, message, (msgX, msgY ), msg_font, msg_scale, msg_color, msg_thickness)

    # Display the grid
    cv2.imshow("Tic Tac Toe Grid", img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

# c_origins = [(3,1),(2,2),(1,3)]
# h_origins = [(2,1),(3,2),(3,3)]

# show_grid(c_origins, h_origins,"X","Your turn!", (2,3))


In [4]:
np.array_equal(None, np.arange(5))

False

In [137]:
# List of G-code for each circle
gcode_circles = [
    #Box 1

"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X61 Y160 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X85 Y160 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X109 Y160 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X61 Y136 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X85 Y136 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X109 Y136 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X61 Y112 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X85 Y112 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X109 Y112 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",

#Box 2

"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X153.5 Y160 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X177.5 Y160 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X201.5 Y160 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X153.5 Y136 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X177.5 Y136 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X201.5 Y136 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X153.5 Y112 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X177.5 Y112 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X201.5 Y112 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
  
#Box 3

"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X246 Y160 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X270 Y160 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X294 Y160 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X246 Y136 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X270 Y136 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X294 Y136 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X246 Y112 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X270 Y112 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X294 Y112 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",

#Box 4

"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X61 Y60 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X85 Y60 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X109 Y60 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X61 Y36 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X85 Y36 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X109 Y36 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X61 Y12 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X85 Y12 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X109 Y12 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",

#Box 5

"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X153.5 Y60 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X177.5 Y60 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X201.5 Y60 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X153.5 Y36 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X177.5 Y36 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X201.5 Y36 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X153.5 Y12 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X177.5 Y12 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X201.5 Y12 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",

#Box 6

"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X246 Y60 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X270 Y60 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X294 Y60 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X246 Y36 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X270 Y36 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X294 Y36 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X246 Y12 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X270 Y12 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n",
"G92 X35 Y200 Z5 \n G1 Z5 F10000 \n G0 X294 Y12 \n G1 Z0 \n G2 I6 \n G1 Z5 F10000 \n G0 X35 Y200 \n M118 E1 #### \n"
]

# Function to retrieve G-code for a specific box and line
def get_gcode(box, line):
    index = (box - 1) * 9 + (line - 1)
    gcode = gcode_circles[index]
    return f"; Box: {box}, Line: {line} \n{gcode}"

# Example of how to retrieve G-code for a specific box and line
# string_input = input("enter circle pos")
# box_number = int(string_input[0])
# line_number = int(string_input[1])
# print(get_gcode(box_number, line_number))

In [7]:

def draw_grid():
    # Open the serial port
    global ser
    
    f = open("./Grid.gcode", "+r")
    data = f.readlines()

    # Send data
    for line in data:
        data_to_send = line
        ser.write(data_to_send.encode('utf-8'))  # Send data as bytes
        time.sleep(0.8)  # Wait for the data to be sent
    # Send data
    data_to_send = "G28\n G0 X35 Y200 Z5 F10000\n"  # Ensure the command ends with \n, or \r\n if needed
    ser.write(data_to_send.encode('utf-8'))  # Send data as bytes

    # Optionally, read the response
    response = ser.readline().decode('utf-8').strip()
    print("Received:", response)

    # Close the serial port

# draw_grid()

In [8]:

def recenter():
    # Open the serial port
    global ser
    # Wait for the serial connection to initialize

    # Send data
    data_to_send = "G28\n G0 X35 Y200 Z5 F10000\n"  # Ensure the command ends with \n, or \r\n if needed
    ser.write(data_to_send.encode('utf-8'))  # Send data as bytes

    # Optionally, read the response
    response = ser.readline().decode('utf-8').strip()
    print("Received:", response)

    # Close the serial port

# draw_grid()

In [145]:

# import serial
# import time
# ser = serial.Serial('COM8', 250000, timeout=1) 
# # Wait for the serial connection to initialize
# time.sleep(2)

def draw_circle(box_number, cordinates):
    global ser
    # Open the serial port
    x = cordinates[0]
    y = cordinates[1]
    line_number = (y-1)*3 + x

    # Wait for the serial connection to initialize
    # time.sleep(2)

    # Send data
    data_to_send = get_gcode(box_number, line_number)
    ser.write(data_to_send.encode('utf-8'))  # Send data as bytes

 

    # Close the serial port

# draw_circle(5, (3,3))

In [168]:
#!/usr/bin/env python3
from math import inf as infinity
from random import choice
import platform
import time
from os import system

"""
An implementation of Minimax AI Algorithm in Tic Tac Toe,
using Python.
This software is available under GPL license.
Author: Clederson Cruz
Year: 2017
License: GNU GENERAL PUBLIC LICENSE (GPL)
"""
isCompTurn = True
first_move = True
future_move = None
HUMAN = +1
COMP = -1
screen_message = ""
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]
render_board = board

def evaluate(state):
    """
    Function to heuristic evaluation of state.
    :param state: the state of the current board
    :return: +1 if the computer wins; -1 if the human wins; 0 draw
    """
    global isCompTurn
    if wins(state, COMP):
        score = +1
    elif wins(state, HUMAN):
        score = -1
    else:
        score = 0

    return score


def wins(state, player):
    """
    This function tests if a specific player wins. Possibilities:
    * Three rows    [X X X] or [O O O]
    * Three cols    [X X X] or [O O O]
    * Two diagonals [X X X] or [O O O]
    :param state: the state of the current board
    :param player: a human or a computer
    :return: True if the player wins
    """
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    if [player, player, player] in win_state:
        return True
    else:
        return False


def game_over(state):
    """
    This function test if the human or computer wins
    :param state: the state of the current board
    :return: True if the human or computer wins
    """
    return wins(state, HUMAN) or wins(state, COMP)


def empty_cells(state):
    """
    Each empty cell will be added into cells' list
    :param state: the state of the current board
    :return: a list of empty cells
    """
    cells = []

    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if cell == 0:
                cells.append([x, y])

    return cells


def valid_move(x, y):
    """
    A move is valid if the chosen cell is empty
    :param x: X coordinate
    :param y: Y coordinate
    :return: True if the board[x][y] is empty
    """
    if [x, y] in empty_cells(board):
        return True
    else:
        return False


def set_move(x, y, player):
    """
    Set the move on board, if the coordinates are valid
    :param x: X coordinate
    :param y: Y coordinate
    :param player: the current player
    """
    if valid_move(x, y):
        board[x][y] = player

        return True
    else:
        return False


def minimax(state, depth, player):
    """
    AI function that choice the best move
    :param state: current state of the board
    :param depth: node index in the tree (0 <= depth <= 9),
    but never nine in this case (see iaturn() function)
    :param player: an human or a computer
    :return: a list with [the best row, best col, best score]
    """
    if player == COMP:
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]

    for cell in empty_cells(state):
        x, y = cell[0], cell[1]
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value

    return best


def clean():
    """
    Clears the console
    """
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')


def render(state, c_choice, h_choice):
    global future_move
    global screen_message
    """
    Print the board on console
    :param state: current state of the board
    """

    chars = {
        HUMAN: h_choice,
        COMP: c_choice,
        0: ' '
    }
    str_line = '---------------'
    h_origins, c_origins = [], []
    print('\n' + str_line)
    for i, row in enumerate(state):
        for j, cell in enumerate(row):
            if cell == HUMAN:
                h_origins.append((j+1, i+1))
            elif cell == COMP:
                c_origins.append((j+1, i+1))
            symbol = chars[cell]
            print(f'| {symbol} |', end='')
        print('\n' + str_line)

    # if(isCompTurn):
    #     screen_message = COMP_Turn_msg
    # else:
    #     screen_message = HUM_Turn_msg
    show_grid(c_origins, h_origins, c_char=c_choice, message=screen_message, future_move= future_move)

def ai_turn(c_choice = 'O', h_choice = 'X'):
    global future_move
    global grid_num
    global first_move
    global isCompTurn
    global render_board
    """
    It calls the minimax function if the depth < 9,
    else it choices a random coordinate.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    if np.sum(np.array(board))==-1:
        future_move = None
        return
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        future_move = None
        return

    clean()
    print(f'Computer turn [{c_choice}]')
    # render(board, c_choice, h_choice)

    if depth == 9 and first_move == True:
        first_move = False
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, COMP)
        x, y = move[0], move[1]
    future_move = (y+1, x+1)

    try:
        render(render_board, c_choice, h_choice)
        time.sleep(1)
        box_number = grid_num + 1
        draw_circle(box_number=box_number, cordinates=future_move)
        print("Drawing! on ", future_move)
    except:
        print("Error drawing circle")
        pass
    # isCompTurn = False
    print(future_move)
    # set_move(x, y, COMP)
    # time.sleep(1)


def human_turn(c_choice, h_choice):
    """
    The Human plays choosing a valid move.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    # Dictionary of valid moves
    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clean()
    print(f'Human turn [{h_choice}]')
    # render(board, c_choice, h_choice)
   
    # while move < 1 or move > 9:
    #     try:
    #         move = int(input('Use numpad (1..9): '))
    #         coord = moves[move]
    #         can_move = set_move(coord[0], coord[1], HUMAN)

    #         if not can_move:
    #             print('Bad move')
    #             move = -1
    #     except (EOFError, KeyboardInterrupt):
    #         print('Bye')
    #         exit()
    #     except (KeyError, ValueError):
    #         print('Bad choice')

threashold = 7
counter = 0
prev_board = []
gameover_counter = 0
threashold_gameover = 7
def main_solve(c_choice='O'):
    #globals
    global threashold
    global counter
    global prev_board
    global isCompTurn
    global screen_message
    global future_move
    global render_board
    global grid_num
    global first_move
    global gameover_counter
    global threashold_gameover
    global start_to_play
    """
    Main function that calls all functions
    """
    clean()
    h_choice = ''  # X or O
    first = 'N'  # if human is the first



    # Setting computer's choice
    if c_choice == 'X':
        h_choice = 'O'
    else:
        h_choice = 'X'

    # Human may starts first
    clean()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Main loop of this game
    
    isOver = False
    if len(empty_cells(board)) > 0 and not game_over(board):
        gameover_counter = 0
        isOver = False
        print(counter)
        if np.array_equal(np.array(prev_board), np.array(board)):
            counter += 1
            
            if counter > threashold:
                counter = 0
                # render the board
                render_board = board
                # render(render_board, c_choice, h_choice)
                if np.sum(np.array(board))==0 and isCompTurn:
                    screen_message = COMP_Turn_msg
                    ai_turn(c_choice, h_choice)
                else:
                    future_move = None
                    screen_message = HUM_Turn_msg

                isCompTurn = not isCompTurn
        else:
            # render the board
            # render(board, c_choice, h_choice)
            # future_move = None
            counter = 0
            # isCompTurn = False
            prev_board = board
        # ai_turn(c_choice, h_choice)

    # Game over message
    
    elif (game_over(board) or len(empty_cells(board)) == 0) and grid_num < 6 and np.array_equal(np.array(prev_board), np.array(board)):
        gameover_counter += 1
        print(gameover_counter)
        if gameover_counter > threashold_gameover:
            gameover_counter = 0
            start_to_play = False
            grid_num = min(grid_num+1, 5)
            first_move = True
            if wins(board, HUMAN):
                if not isOver:
                    render_board = board
                isOver = True
                clean()
                print(f'Human turn [{h_choice}]')
                screen_message = win_msg
                # render(board, c_choice, h_choice)
                print('YOU WIN!')
            elif wins(board, COMP):
                if not isOver:
                    render_board = board
                isOver = True        
                clean()
                print(f'Computer turn [{c_choice}]')
                screen_message = lose_msg
                # render(board, c_choice, h_choice)
                print('YOU LOSE!')
            else:
                if not isOver:
                    render_board = board
                isOver = True
                clean()
                # render(board, c_choice, h_choice)
                print('DRAW!')
                screen_message = draw_msg
        else:
            # gameover_counter = 0
            pass
    elif not np.array_equal(np.array(prev_board), np.array(board)):
        prev_board = board
    # render the board
    render(render_board, c_choice, h_choice)
    # exit()


if __name__ == '__main__':
    # main()
    pass

In [170]:
import serial
import time
ser = serial.Serial('COM8', 250000, timeout=1) 
# Wait for the serial connection to initialize
time.sleep(2)

import json
# from IPython.display import clear_output
start_to_play = False
computer_choice = "O"
grid_num = 0
# box_number = grid_num + 1

# now let's initialize the list of reference point
ref_points = [[[]]*2 for i in range(6)]
try:
    with open('ref_points.json', 'r') as file:
        ref_points = json.load(file)
except RuntimeError as e:
    print("Error: ", e)
draw = False
def shape_selection(event, x, y, flags, param):
    # grab references to the global variables
    global ref_points, crop, draw
    ref_point = ref_points[grid_num]
    # ix, iy = (0, 0)
    # if the left mouse button was clicked, record the starting
    # (x, y) coordinates and indicate that cropping is being performed
    if event == cv2.EVENT_LBUTTONDOWN:
        draw = True
        ref_point[0] = (x, y)
        ref_point[1] = ()
        # image = clone.copy()
        # cv2.imshow("image", image)

    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates and indicate that
        # the cropping operation is finished
        if draw:
            draw = False
            ref_point[1] = (x, y)
            ix, iy = (ref_point[0][0], ref_point[0][1])
            # calculate the side length of the square
            side_length = max(abs(x - ix), abs(y - iy))

            # determine the bottom-right corner of the square
            if x >= ix and y >= iy:
                ref_point[1] = (ix + side_length, iy + side_length)
            elif x >= ix and y < iy:
                ref_point[1] = (ix + side_length, iy - side_length)
            elif x < ix and y >= iy:
                ref_point[1] = (ix - side_length, iy + side_length)
            else:
                ref_point[1] = (ix - side_length, iy - side_length)

                        
            # cv2.imshow("image", image)




# load the image, clone it, and setup the mouse callback function
ret_val, image = cam.read()
# image = cv2.imread('download.png')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", shape_selection)

croped_img = None
# keep looping until the 'q' key is pressed
rotation = 2
prev_state = None
img_counter = 0
n_imgs = 1
croped_imgs = []
while True:
    # display the image and wait for a keypress
    ret_val, image = cam.read()
    for _ in range(rotation):
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    ref_point = ref_points[grid_num]
    if len(ref_point[1])==2:
        ix, iy = (ref_point[0][0], ref_point[0][1])
        x, y = (ref_point[1][0], ref_point[1][1])
        # calculate the side length of the square
        side_length = max(abs(x - ix), abs(y - iy))
        if side_length >= 64:
            # draw a square around the region of interest
            thickness = 2
            cv2.rectangle(image, 
                    (ref_point[0][0] - thickness, ref_point[0][1] - thickness), 
                    (ref_point[1][0] + thickness, ref_point[1][1] + thickness), 
                    (0, 255, 0), thickness)
            croped_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            croped_img = croped_img[ref_point[0][1]:ref_point[1][1], ref_point[0][0]:ref_point[1][0]]
            croped_img = cv2.resize(croped_img, (64, 64))
            croped_img = cv2.adaptiveThreshold(croped_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11,2)
            croped_img = 255-croped_img
            croped_img = croped_img.astype(np.float64)
            croped_img /= 255

            clear_output(wait=True)
            cv2.imshow("cropped", cv2.resize(croped_img, (256, 256)))
            # plt.imshow(croped_img, cmap=plt.cm.gray)

            # state = np.round(d.predict(croped_img))
            # if np.array_equal(state, prev_state):
            #     prev_state = state
            #     counter = 0
            # else :
            #     counter += 1
            if img_counter < n_imgs:
                croped_imgs.append(croped_img)
                img_counter += 1
            else:
                state = np.round(d.predict_average(croped_imgs))
                img_counter = 0
                croped_imgs = []
                print(state)
                state = ones_state(state)
                if computer_choice == "X":
                    state = state * -1
                board = [state[i:i + 3] for i in range(0, len(state), 3)]
                if start_to_play == True:
                    main_solve(c_choice=computer_choice)
                
    
    cv2.imshow("image", image)
    
    key = cv2.waitKey(1) & 0xFF

    # press 'r' to reset the window
    if key == ord("r"):
        # draw = True
        rotation += 1
        rotation %= 4
    elif key >= ord("1") and (key - ord("1")) < 6:
        grid_num_to_print = key - ord("1") + 1
        print(grid_num_to_print)
        grid_num = key - ord("1")
    elif key == ord("g"):
        try:
            draw_grid()
        except:
            print("error drawing the grid")
    elif key == ord("y"):
        try:
            recenter()
        except RuntimeError as e:
            print("error: ",e)
    elif key == ord("s"):
        #start playing
        start_to_play = True
        
    # if the 'c' key is pressed, break from the loop
    elif key == 27:
        break
if len(ref_points)==6:
    with open('ref_points.json', 'w') as json_file:
        json.dump(ref_points, json_file) 
    print('ref_points.json updated!')
# close all open windows
print(ref_point)
ser.close()
cv2.destroyAllWindows() 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1.]
ref_points.json updated!
[[342, 327], [451, 436]]
